In [1]:
import pandas as pd
import numpy as np
from tokenizers import BertWordPieceTokenizer

In [2]:
sentences = pd.read_csv('./data/bookraw_list.txt')
sentence_list = sentences.iloc[:,0]

# nan 제거
sentence_list = sentence_list[sentence_list.isna() == False].values

# to numpy

### token 상위 99% 이상인 문장 제거 
num_sen = [len(sen.split(' ')) for sen in sentence_list.tolist()]

percentile_99 = np.quantile(num_sen,0.99)

sentence_list_under_99 = sentence_list[np.array(num_sen) < percentile_99 ].tolist()

print(f"최대 토큰 개수 {percentile_99}")

최대 토큰 개수 30.0


In [2]:
## Tokenizer 불러오기
## 한글을 불러올 땐 strip accents = False 필수

vocab = './data/vocab.txt'
tokenizer = BertWordPieceTokenizer.from_file(vocab=vocab,strip_accents=False)


encoded = tokenizer.encode('python과 javascript를 만들며 고민했다')
print(encoded.tokens)

['[CLS]', 'python', '##과', 'javascript', '##를', '만들며', '고민', '##했다', '[SEP]']


### mask_token 구현

In [ ]:

# 1) 문장 변형은 Token 중 15%
# 2) 15% 변형 중 80%는 mask, 20%는 단어 바꾸기임.
import random

def mask_sentence(sentence : list, ids : list ,tokenizer, mask_rate = 0.15) :
    '''
    sentence : ['[CLS]', '피벗', '테이블에', '원본', '데이터의', '변경된', '값', '적용하기', '[SEP]']
    ids : [2, 4537, 11429, 7412, 4081, 12359, 84, 3604, 3]
    '''

    sentence = sentence.copy()
    ids = ids.copy()

    # token 개수
    len_sen = len(sentence)

    # token_mask 
    token_mask = [True for _ in range(len_sen)]

    # -2 하는 이유 [CLS], [SEP] 제외
    mask_amount = round(len_sen-2 * mask_rate)

    for _ in range(mask_amount) :
        i = random.randint(1,len_sen-2)

        if random.random() < 0.8 :
            sentence[i] ='[MASK]'
            ids[i] = 4
        else : 
            # 5부터 선정하는 이유 
            # [PAD] = 0, [UNK] = 1, [CLS] = 2, [SEP] = 3, [MASK] = 4

            j = random.randint(5,tokenizer.get_vocab_size())
            sentence[i] = tokenizer.decode([j])
            ids[i] = j

        token_mask[i] = False

        return sentence, ids ,token_mask




def padding_sentence(mask_sentence: tuple,max_length = 40) :
    sentence, ids ,token_mask = mask_sentence
    len_sen = len(sentence)
    sentence.extend(['[PAD]' for _ in range(max_length-len_sen)])
    ids.extend([0 for _ in range(max_length-len_sen)])
    token_mask.extend([False for _ in range(max_length-len_sen)])

    return sentence, ids ,token_mask


### token 구현한 것 실현

In [245]:
max_length = 40 

def tokenizing_sentence(sent:str, max_length) : 

    encoded = tokenizer.encode(sent)
    
    sen_token = encoded.tokens
    sen_ids = encoded.ids
    
    if len(sen_token) > max_length : 
        max_length = len(sen_token)

    # semtemce, ids, token_mask
    mss = mask_sentence(sen_token,sen_ids,tokenizer=tokenizer)

    # 패딩 구현
    sentence, ids ,token_mask = padding_sentence(mss)
        
    return sentence, ids ,token_mask

print(tokenizing_sentence(sent,max_length=max_length))

(['[CLS]', '피벗', '테이블에', '원본', '데이터의', '변경된', '[MASK]', '적용하기', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]'], [2, 4537, 11429, 7412, 4081, 12359, 4, 3604, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [True, True, True, True, True, True, False, True, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False])


### NSP

NSP를 먼저 만든 다음

그 안에서 MLM을 구성하는게 맞다. 



In [221]:
total = pd.read_csv('./data/bookraw_total.csv',index_col=0)

for row in total.iloc[:,1:].iterrows() :
    row = row[1]

print(row)

book_toc      ['서문', '베타 리더 추천사', '이 책의 구성', '장 데이터 사이언티스트 이...
book_intro    ['', '데이터 사이언티스트의 실제 업무를 알려드리고 데이터 사이언티스트가 되기 ...
publisher     ['데이터 사이언티스트가 되기 위해서라면 가장 먼저 풀어야 할 데이터 사이언티스트의...
Name: 5067, dtype: object


In [232]:
import ast
def changeStringToList(strList:str):
    """

    dataframe안에 list를 통으로 넣으면 str으로 저장된다.
    ast 라이브러리를 쓰면 원래 ㅣist 이지만 str 타입으로 표현된 값을 다시 list 타입으로 바꿔준다.

    """
    return ast.literal_eval(strList)

    

### 도서 하나에 대한 모든 문장을 넣는 매소드 

In [267]:

def merge_row_items(row:pd.Series) : 
    '''
    하나의 row에 있는 column을 통합하는 매소드
    '''
    str_list = row.tolist()

    #'['asd']' => ['asd']
    section_list = list(map(changeStringToList,str_list))

    b = []
    for a in section_list :
        b += a

    b = list(filter(None,b))
    return b



row_test = merge_row_items(row)[10:12]
max_length = 40
for i in range(0,len(row_test)-1) : 
    # 오리지널 버전 
    first = tokenizer.encode(row_test[i])
    
    second = tokenizer.encode(row_test[i+1])

    # mask, 단어 변경된 버전 
    first = tokenizing_sentence(row_test[i],max_length=40)
    second = tokenizing_sentence(row_test[i+1],max_length=40)
    # 첫번째, 두번째 문장 합치기
    sen,ids,mask = [first[i]+second[i][1:] for i in range(3)]
    

    


['[CLS]', '데이터', '사이언티스트', '##를', '희망한다', '##면', '[MASK]', '##부터', '살펴', '##라', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '관련', '전공자', '##와', '석', '박사', '씻', '우대', '##하는', '현실', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]'] 79


### 현상황 정리

배경
1. NSP를 먼저 구성한 다음 MLM을 하는 식으로 구성해야함.
2. NSP 구성 시 원본버전과 mask 버전 두개를 만들어야하는데 mask 버전을 먼저 만들다보니 복잡하게 구성됨

할 일

3. 원본 버전과 마스크 버전 매서드 개선하기
4. class로 새롭게 구현하기.